# Tarea 2 - Recuperación de Información Multimedia
El siguiente reporte pretende contestar las preguntas planteadas en el enunciado. Para esto se deben cargar los descriptores, y analizar los datos aplicando las técnicas vistas en clases.

## Carga de Descriptores

In [3]:
import numpy
import os


def load_file(filename, num_vectors, vector_dimensions):
    assert os.path.isfile(filename), "no existe archivo " + filename
    mat = numpy.fromfile(filename, dtype=numpy.float32)
    return numpy.reshape(mat, (num_vectors, vector_dimensions))


def load_dataset_pair(dirname, num_vectors_q, num_vectors_r, vector_dimensions):
    file_q = "{}/Q-{}_{}_4F.bin".format(dirname, num_vectors_q, vector_dimensions)
    file_r = "{}/R-{}_{}_4F.bin".format(dirname, num_vectors_r, vector_dimensions)
    data_q = load_file(file_q, num_vectors_q, vector_dimensions)
    data_r = load_file(file_r, num_vectors_r, vector_dimensions)
    return data_q, data_r


(dataset_q, dataset_r) = load_dataset_pair("descriptores/MEL128", 21573, 33545, 128)
print("Q={} R={}".format(dataset_q.shape, dataset_r.shape))

(dataset_q, dataset_r) = load_dataset_pair("descriptores/SIFT", 2886, 202088, 128)
print("Q={} R={}".format(dataset_q.shape, dataset_r.shape))

(dataset_q, dataset_r) = load_dataset_pair("descriptores/VGG19", 842, 10171, 4096)
print("Q={} R={}".format(dataset_q.shape, dataset_r.shape))

Q=(21573, 128) R=(33545, 128)
Q=(2886, 128) R=(202088, 128)
Q=(842, 4096) R=(10171, 4096)


## Construcción y Aplicación de Índices

In [4]:
import pyflann
import time

#crea un objeto flann
flann = pyflann.FLANN()

### Linear Scan

In [30]:
#construir el indice linear scan
t0 = time.time()
flann.build_index(dataset_r, algorithm="linear")
t1 = time.time()
print("construccion linear scan={:.1f}".format(t1-t0))

#buscar el NN usando el ultimo indice construido (linear scan)
t0 = time.time()
lscan_results, lscan_dists = flann.nn_index(dataset_q, num_neighbors=1, cores=1)
t1 = time.time()
tiempo_lscan = t1-t0
print("busqueda linear scan={:.1f}".format(tiempo_lscan))

construccion linear scan=0.0
busqueda linear scan=58.3


### KD-Tree

In [40]:
def evaluar_resultado(results, dists, tiempo):
    correctas = 0
    incorrectas = 0
    iguales = 0
    for i in range(len(lscan_results)):
        #comparar las distancias
        if dists[i] == lscan_dists[i]: 
            correctas += 1
        elif dists[i] > lscan_dists[i]: 
            incorrectas += 1        
        else:
            assert False, "distancia erronea!"
        #comparar el NN
        if results[i] == lscan_results[i]: 
            iguales += 1
    efectividad = 100 * correctas / (correctas + incorrectas)
    eficiencia = 100 * tiempo / tiempo_lscan
    print("efectividad={:.1f}% eficiencia={:.1f}% correctas={} incorrectas={} mismo-NN={}".format(efectividad, eficiencia, correctas, incorrectas, iguales))
    return efectividad, eficiencia, correctas, incorrectas, iguales

In [44]:
def aplicar_kdtree(trees, checks):
    #construir el indice kdtree con trees = trees
    t0 = time.time()
    flann.build_index(dataset_r, algorithm="kdtree", trees=trees)
    t1 = time.time()
    print("construccion kdtree={:.1f}, trees={}".format(t1-t0, trees))

    #buscar aproximada del NN con el ultimo indice construido (kdtree)
    t0 = time.time()
    approx_results, approx_dists = flann.nn_index(dataset_q, num_neighbors=1, cores=1, checks=checks)
    t1 = time.time()
    tiempo = t1-t0
    print("busqueda aproximada kdtree={:.1f}, trees={}".format(tiempo, trees))

    efectividad, eficiencia, correctas, incorrectas, iguales = evaluar_resultado(approx_results, approx_dists, tiempo)
    return efectividad, eficiencia, correctas, incorrectas, iguales

In [45]:
efectividad, eficiencia, correctas, incorrectas, iguales = aplicar_kdtree(1, 100)

construccion kdtree=2.0, trees=1
busqueda aproximada kdtree=0.6, trees=1
efectividad=63.9% eficiencia=1.1% correctas=538 incorrectas=304 mismo-NN=538


In [47]:
efectividad, eficiencia, correctas, incorrectas, iguales = aplicar_kdtree(1, 1000)

construccion kdtree=2.0, trees=1
busqueda aproximada kdtree=6.3, trees=1
efectividad=90.5% eficiencia=10.7% correctas=762 incorrectas=80 mismo-NN=761


In [48]:
efectividad, eficiencia, correctas, incorrectas, iguales = aplicar_kdtree(2, 100)

construccion kdtree=4.0, trees=2
busqueda aproximada kdtree=0.7, trees=2
efectividad=65.8% eficiencia=1.1% correctas=554 incorrectas=288 mismo-NN=554


In [49]:
efectividad, eficiencia, correctas, incorrectas, iguales = aplicar_kdtree(2, 1000)

construccion kdtree=3.9, trees=2
busqueda aproximada kdtree=6.3, trees=2
efectividad=91.8% eficiencia=10.9% correctas=773 incorrectas=69 mismo-NN=773
